In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, models
from facenet_pytorch import MTCNN
import cv2
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model
class DeepFakeModel(nn.Module):
    def __init__(self):
        super(DeepFakeModel, self).__init__()
        self.model = models.resnet18(pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, 2)  # 2 classes: Real, Fake

    def forward(self, x):
        return self.model(x)

    def load_state_dict(self, state_dict, strict=True):
        # Create a new state_dict with the 'model.' prefix
        new_state_dict = {}
        for k, v in state_dict.items():
            new_state_dict['model.' + k] = v
        super().load_state_dict(new_state_dict, strict=strict)

# Initialize and load model
model = DeepFakeModel()  # Instantiate the model
model.load_state_dict(torch.load(r"C:\Users\welcome\deepfake_model.pth", map_location=device))
model.to(device)
model.eval()


# Transformation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

# Face extractor
mtcnn = MTCNN(image_size=160, margin=20, keep_all=True, post_process=True)

def extract_faces_from_video(video_path, skip_frames=5):
    cap = cv2.VideoCapture(video_path)
    faces = []
    frame_num = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_num += 1
        if frame_num % skip_frames != 0:
            continue

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        detected_faces = mtcnn(rgb_frame)

        if detected_faces is None:
            continue

        if isinstance(detected_faces, torch.Tensor) and detected_faces.ndim == 3:
            detected_faces = [detected_faces]

        for face in detected_faces:
            face = torch.clamp(face, 0, 1)
            face_np = face.permute(1, 2, 0).detach().cpu().numpy()
            face_np = (face_np * 255).astype(np.uint8)
            face_tensor = transform(face_np)
            faces.append(face_tensor)

    cap.release()
    return faces

# Predict video
def predict_video(video_path):
    faces = extract_faces_from_video(video_path)
    if len(faces) == 0:
        print("[ERROR] No faces found.")
        return

    inputs = torch.stack(faces).to(device)
    with torch.no_grad():
        outputs = model(inputs)
        predictions = torch.argmax(outputs, dim=1)
        predicted_classes = predictions.cpu().numpy()

        # Majority vote
        real_votes = np.sum(predicted_classes == 0)
        fake_votes = np.sum(predicted_classes == 1)
        print(f"[INFO] Real votes: {real_votes}, Fake votes: {fake_votes}")

        if fake_votes > real_votes:
            print("🔴 Final Prediction: FAKE")
        else:
            print("🟢 Final Prediction: REAL")

# Example usage
video_path =  r"E:\predict\MyVideo.mkv" # Replace with your test video path
predict_video(video_path)

C:\Users\welcome\anaconda3\envs\deepfake_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\welcome\anaconda3\envs\deepfake_env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\welcome\anaconda3\envs\deepfake_env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[INFO] Real votes: 70, Fake votes: 107
🔴 Final Prediction: FAKE
